<a href="https://colab.research.google.com/github/ericjenn/working-groups/blob/ericjenn-srpwg-wg1/safety-related-profile/documents/profile_opset/div/div.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:

#!pip install onnx onnxruntime


In [10]:
import onnx
import onnxruntime as ort
import numpy as np

# Define input and output tensor names
input_name = "X"
sigmoid_output_name = "SigmoidOutput"

#-------------------------------------------------------------------------------
# 2-rank input tensor
#-------------------------------------------------------------------------------

# Create 2-rank input tensor
input_tensor = onnx.helper.make_tensor_value_info(input_name, onnx.TensorProto.FLOAT, [None, None])

# Create output tensor (final result after sigmoid operation)
sigmoid_output = onnx.helper.make_tensor_value_info(sigmoid_output_name, onnx.TensorProto.FLOAT, [None, None])

# Define Sigmoid node: Y = Sigmoid(X)
sigmoid_node = onnx.helper.make_node("Sigmoid", [input_name], [sigmoid_output_name])

# Create the ONNX graph
graph = onnx.helper.make_graph(
    nodes=[sigmoid_node],
    name="Sigmoid",
    inputs=[input_tensor],
    outputs=[sigmoid_output]
)

# Create the ONNX model (Sigmoid available since early opsets, 13 is safe)
model = onnx.helper.make_model(graph, opset_imports=[onnx.helper.make_opsetid("", 13)]) # Explicitly set opset to 13
onnx.checker.check_model(model)

# Save the model
onnx.save(model, "sigmoid.onnx")

# Load and run the model using ONNX Runtime
session = ort.InferenceSession("sigmoid.onnx")

def do_sigmoid(x):
    # Run inference
    output = session.run(None, {input_name: x})

    x_f = np.array2string(x, separator=',', max_line_width=np.inf).replace('\n', '')
    o_f = np.array2string(output[0], separator=',', max_line_width=np.inf).replace('\n', '')

    # Display results
    print(f"X={x_f}")
    print(f"Result: sigmoid(X)={o_f}")

np.set_printoptions(precision=None, floatmode='fixed')


#--------------------------------------------------------------------
# Nominal cases
#--------------------------------------------------------------------

# Case N1: 2-rank tensor (float32)
x = np.array([[1.0, 2.0],
              [3.0, 4.0]], dtype=np.float32)

do_sigmoid(x)

#--------------------------------------------------------------------
# Non nominal cases (nan, -inf, 0, negative values)
#--------------------------------------------------------------------

v = [1.0, 0.0, -1.0, float("inf"), float("-inf"), float("nan")]

for x_val in v:
    x_np = np.array([[x_val],
                     [x_val]], dtype=np.float32)
    do_sigmoid(x_np)



#--------------------------------------------------------------------
# Examples from documentation
#--------------------------------------------------------------------

print("\n## Example 1")

# Example 1
# X = [0, 1, -1]
x_example_1 = np.array([[0.0, 1.0, -1.0]], dtype=np.float32)
do_sigmoid(x_example_1)

print("\n## Example 2")

# Example 2
# X =
# [[  -2   0 ]
#  [  1    2  ]
#  [ -4    4   ]]
x_example_2 = np.array([
    [-2.0,   0.0],
    [1.0,    2.0],
    [-4.0,   4.0]
], dtype=np.float32)

do_sigmoid(x_example_2)

print("\n## Example 3")

# Example 3
# X =
# [[  inf   nan   -inf]
x_example_3 = np.array([
    [float("inf"),   float("nan"), float("-inf")]
], dtype=np.float32)

do_sigmoid(x_example_3)

X=[[1.00000000,2.00000000], [3.00000000,4.00000000]]
Result: sigmoid(X)=[[0.73105860,0.88079709], [0.95257413,0.98201376]]
X=[[1.00000000], [1.00000000]]
Result: sigmoid(X)=[[0.73105860], [0.73105860]]
X=[[0.00000000], [0.00000000]]
Result: sigmoid(X)=[[0.50000000], [0.50000000]]
X=[[-1.00000000], [-1.00000000]]
Result: sigmoid(X)=[[0.26894143], [0.26894143]]
X=[[inf], [inf]]
Result: sigmoid(X)=[[1.00000000], [1.00000000]]
X=[[-inf], [-inf]]
Result: sigmoid(X)=[[0.00000000], [0.00000000]]
X=[[nan], [nan]]
Result: sigmoid(X)=[[nan], [nan]]

## Example 1
X=[[ 0.00000000, 1.00000000,-1.00000000]]
Result: sigmoid(X)=[[0.50000000,0.73105860,0.26894143]]

## Example 2
X=[[-2.00000000, 0.00000000], [ 1.00000000, 2.00000000], [-4.00000000, 4.00000000]]
Result: sigmoid(X)=[[0.11920291,0.50000000], [0.73105860,0.88079709], [0.01798624,0.98201376]]

## Example 3
X=[[ inf, nan,-inf]]
Result: sigmoid(X)=[[1.00000000,       nan,0.00000000]]
